# Model development

In [1]:
from src.models.model import *

# Create an NER model from a base pretrained model
model_loader = ModelLoader('UFNLP/gatortron-base')
NER_model, tokenizer, config = model_loader.load_model(num_labels = 10)

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/Tutorial-LLM/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of MegatronBertForTokenClassification were not initialized from the model checkpoint at UFNLP/gatortron-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas